# Modelo preditivo para a tendência do IBOVESPA

O objetivo desse projeto é desenvolver um modelo de Machine Learning capaz de prever se o índice IBOVESPA vai fechar em alta ou baixa no dia seguinte, com acuracidade mínima de 75% em um conjunto de teste composto pelos últimos 30 dias de dados.

> O Ibovespa é o principal indicador de desempenho das ações negociadas na B3 e reúne as empresas mais importantes do mercado de capitais brasileiro. Foi criado em 1968 e, ao longo desses 50 anos, consolidou-se como referência para investidores ao redor do mundo. [Referência](https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/ibovespa.htm)

## Importação das bibliotecas

In [78]:
import pandas as pd
import numpy as np
import talib

## Aquisição dos dados

Utilizaremos os dados históricos do índice IBOVESPA, disponíveis publicamente no site do [br.investing](https://br.investing.com/indices/bovespa-historical-data)

In [79]:
input_path = '../data/raw/dados_historicos_ibovespa_2008-2025.csv'

df = pd.read_csv(input_path, thousands='.', decimal=',', parse_dates=['Data'], date_format='%d.%m.%Y', index_col='Data')
df = df.rename_axis('ds').sort_index()
df.tail()

,Último,Abertura,Máxima,Mínima,Vol.,Var%
ds,,,,,,
2025-06-12,137800,137127,137931,136175,"7,12B","0,49%"
2025-06-13,137213,137800,137800,136586,"8,63B","-0,43%"
2025-06-16,139256,137212,139988,137212,"7,62B","1,49%"
2025-06-17,138840,139256,139497,138293,"8,38B","-0,30%"
2025-06-18,138717,138844,139161,138443,"8,32B","-0,09%"


In [80]:
df.shape

(4315, 6)

In [81]:
# informações gerais do dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Último    4315 non-null   int64 
 1   Abertura  4315 non-null   int64 
 2   Máxima    4315 non-null   int64 
 3   Mínima    4315 non-null   int64 
 4   Vol.      4314 non-null   object
 5   Var%      4315 non-null   object
dtypes: int64(4), object(2)
memory usage: 236.0+ KB


## Tratamento dos dados

Vamos transformar os dados brutos em um formato adequado para o treinamento de um modelo de Machine Learning.

In [82]:
# renomeando as colunas para os nomes padrões utilizados no mercado financeiro
colunas = {
  'Último': 'close',              # fechamento da negociação diária
  'Abertura': 'open',             # início da negociação diária
  'Máxima': 'high',               # valor máximo do dia
  'Mínima': 'low',                # valor mínimo do dia
  'Vol.': 'volume',               # volume de negociação diária
  'Var%': 'daily_return'          # variação percentual diária
}

df.rename(columns=colunas, inplace=True)

In [83]:
# Data mínima, máxima e total de anos do DF levando em conta os anos bissextos
print(f"Os dados vão de {df.index.min().date()} até {df.index.max().date()}, o que dá aproximadamente {(df.index.max() - df.index.min()).days / 365.25:.0f} anos")

Os dados vão de 2008-01-18 até 2025-06-18, o que dá aproximadamente 17 anos


In [84]:
# conferindo se há valores duplicados
df.duplicated().sum()

np.int64(0)

In [85]:
# conferindo se há valores nulos
df.isnull().sum()

close           0
open            0
high            0
low             0
volume          1
daily_return    0
dtype: int64

In [86]:
df[df['volume'].isnull()]

,close,open,high,low,volume,daily_return
ds,,,,,,
2016-02-10,40377,40592,40592,39960,NaN,"-0,53%"


In [87]:
# ajustando a coluna volume (de texto para float)
def converter_volume(vol):
    if pd.isna(vol):    
        return np.nan
    vol = vol.upper().replace(',', '.').strip()
    if vol.endswith('B'):
        return float(vol[:-1]) * 1e9
    elif vol.endswith('M'):
        return float(vol[:-1]) * 1e6
    elif vol.endswith('K'):
        return float(vol[:-1]) * 1e3
    else:
        return float(vol)

df['volume'] = df['volume'].apply(converter_volume)

In [88]:
# Substituir o volume nulo pela média do volume daquele mês
media_volume = df.loc['2016-02', 'volume'].mean().round(1)
df.fillna(media_volume, inplace=True)

In [89]:
# ajustando a coluna variação percentual diária, que contém o pct_change() do fechamento
df['daily_return'] = df['daily_return'].str.replace('%', '').str.replace(',', '.')
df['daily_return'] = round(df['daily_return'].astype(float) / 100, 4)
df.head()

,close,open,high,low,volume,daily_return
ds,,,,,,
2008-01-18,57506,57039,58291,56241,5810000.0,0.0082
2008-01-21,53709,57503,57503,53487,3570000.0,-0.0660
2008-01-22,56097,53705,56541,53610,3650000.0,0.0445
2008-01-23,54235,56098,56098,53011,3720000.0,-0.0332
2008-01-24,57463,54242,57675,54242,3800000.0,0.0595


In [90]:
# conferindo formato dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4315 entries, 2008-01-18 to 2025-06-18
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   close         4315 non-null   int64  
 1   open          4315 non-null   int64  
 2   high          4315 non-null   int64  
 3   low           4315 non-null   int64  
 4   volume        4315 non-null   float64
 5   daily_return  4315 non-null   float64
dtypes: float64(2), int64(4)
memory usage: 236.0 KB


## Engenharia de atributos

Primeiro, vamos criar a **variável alvo** com horizonte de 1 dia para que o modelo de classificação consiga prever se o fechamento de amanhã será maior que o de hoje.

* `1` para dias de **Alta**
* `0` para dias de **Baixa**

In [91]:
# criando uma coluna de direção (baixa = 0 ou alta = 1)
target_values = (df['close'].shift(-1) > df['close']).astype(int)
df.insert(0, 'target', target_values)

Agora, vamos criar **variáveis preditoras** através de uma análise técnica do mercado financeiro, ao invés de uma análise fundamentalista. Em vez de dar ao modelo os dados brutos de abertura, máxima e mínima, vamos criar features para extrair a informação das relações entre eles.

Para as janelas de tempo, vamos utilizar as convenções do mercado financeiro para representar uma semana (5 dias), um mês (21 dias) e um trimestre (63 dias) de transações.

In [92]:
# indicadores de momentum

# retorno percentual de 1 a 5 dias (tendência de curto prazo)
for lag in range(1, 6):
    df[f'return_lag_{lag}'] = df['daily_return'].shift(lag)

# retorno percentual acumulado de uma semana, um mês e um trimestre
for period in [5, 21, 63]:
    df[f'momentum_{period}'] = df['close'].pct_change(period)

In [93]:
# indicadores de tendência

# média móvel simples de um mês (tendência de médio prazo)
df['sma_21'] = talib.SMA(df['close'], timeperiod=21)

# média móvel exponencial de um trimestre (tendência de longo prazo)
df['ema_50'] = talib.EMA(df['close'], timeperiod=50)

Para a média móvel exponencial, é uma convenção utilizar 50 dias ou 200 dias para tendências de longo prazo. [Referência](https://www.investopedia.com/terms/e/ema.asp#citation-4)

In [94]:
# indicadores de osciladores

# índice de força relativa (IFR) varia de 0 a 100
df['rsi_14'] = talib.RSI(df['close'], timeperiod=14)

Por padrão, utiliza-se um período de 14 dias para o IFR. [Referência](https://blog.quantinsti.com/rsi-indicator/)

In [95]:
# indicadores de volatilidade

# average true range
df['atr_14'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)

Por padrão, utiliza-se um período de 14 dias para o ATR. [Referência](https://www.investopedia.com/terms/a/atr.asp)

Por padrão, utiliza-se um período de 20 dias (um mês) para e 2 desvios-padrão para as bandas para conter cerca de 95% dos dados. [Referência](https://www.infomoney.com.br/guias/bandas-bollinger/)

In [96]:
# indicadores de volume

# on-balance volume 
df['obv'] = talib.OBV(df['close'], df['volume'])

Decidimos não usar as features de volume, uma vez que são dados com muito ruído e que vão de uma escala de mil até bilhão. (vide notebook EDA)

In [97]:
# indicadores de data

# dia da semana (segunda=0 a sexta=4)
df['day_of_week'] = df.index.dayofweek

# dia do mês (1 a 30 ou 31)
df['day_of_month'] = df.index.day

# mês (janeiro=1 a dezembro=12)
df['month'] = df.index.month

# ano (pode não ser bom para previsões de curto prazo)
# df['year'] = df.index.year

## Exportação dos dados

In [98]:
# selecionando apenas 10 anos de dados
df_10years = df.loc[df.index >= '2015-06-17']
df_10years.head()

,target,close,open,high,low,volume,daily_return,return_lag_1,return_lag_2,return_lag_3,...,momentum_21,momentum_63,sma_21,ema_50,rsi_14,atr_14,obv,day_of_week,day_of_month,month
ds,,,,,,,,,,,,,,,,,,,,,
2015-06-17,1,53249,53698,53755,52965,3090000.0,-0.0084,0.0106,-0.0039,-0.0064,...,-0.052576,0.090096,53785.285714,53919.575856,44.356672,889.081392,78500090.0,2,17,6
2015-06-18,0,54239,53251,54352,53214,2750000.0,0.0186,-0.0084,0.0106,-0.0039,...,-0.022703,0.078632,53725.285714,53932.102293,51.525407,906.861293,81250090.0,3,18,6
2015-06-19,1,53749,54236,54236,53479,2950000.0,-0.0090,0.0186,-0.0084,0.0106,...,-0.020983,0.043143,53670.428571,53924.921811,48.214460,896.371201,78300090.0,4,19,6
2015-06-22,0,53864,53750,54342,53655,2430000.0,0.0021,-0.0090,0.0186,-0.0084,...,-0.022645,0.057110,53611.000000,53922.532720,49.042076,881.416115,80730090.0,0,22,6
2015-06-23,1,53772,53865,54361,53772,2710000.0,-0.0017,0.0021,-0.0090,0.0186,...,-0.011126,0.034734,53582.190476,53916.629476,48.375998,860.529250,78020090.0,1,23,6


In [99]:
df_10years.isnull().sum()

target          0
close           0
open            0
high            0
low             0
volume          0
daily_return    0
return_lag_1    0
return_lag_2    0
return_lag_3    0
return_lag_4    0
return_lag_5    0
momentum_5      0
momentum_21     0
momentum_63     0
sma_21          0
ema_50          0
rsi_14          0
atr_14          0
obv             0
day_of_week     0
day_of_month    0
month           0
dtype: int64

In [100]:
# Salvar o CSV já limpo e processado com a data no índice
output_path = '../data/processed/dados_historicos_ibovespa_2015-2025_processed.csv'

df_10years.to_csv(output_path, index=True)